In [1]:
import polars as pl

pl.Config.set_tbl_rows(-1)         # Exibe todas as linhas
pl.Config.set_fmt_str_lengths(500) # Ajusta o número de caracteres por célula
pl.Config.set_tbl_width_chars(200) # Ajusta a largura da tabela no terminal
pl.Config.set_tbl_cols(-1) # Exibe todas as colunas

path_job = '../data/curated/jobs.parquet'
path_applicants = '../data/curated/applicants.parquet'
path_prospects = '../data/curated/prospects.parquet'

df_job = pl.read_parquet(path_job)
df_applicants = pl.read_parquet(path_applicants)
df_prospects = pl.read_parquet(path_prospects)

In [3]:
print(df_prospects.columns)
print(df_applicants.columns) 
print(df_job.columns)

['codigo_vaga', 'titulo', 'modalidade', 'p_nome', 'p_codigo', 'p_situacao_candidado', 'p_data_candidatura', 'p_ultima_atualizacao', 'p_comentario', 'p_recrutador']
['codigo_candidato', 'ib_telefone_recado', 'ib_telefone', 'ib_objetivo_profissional', 'ib_data_criacao', 'ib_inserido_por', 'ib_email', 'ib_local', 'ib_sabendo_de_nos_por', 'ib_data_atualizacao', 'ib_codigo_profissional', 'ib_nome', 'ip_data_aceite', 'ip_download_cv', 'ip_nome', 'ip_cpf', 'ip_fonte_indicacao', 'ip_email', 'ip_email_secundario', 'ip_data_nascimento', 'ip_telefone_celular', 'ip_telefone_recado', 'ip_sexo', 'ip_estado_civil', 'ip_pcd', 'ip_endereco', 'ip_skype', 'ip_url_linkedin', 'ip_facebook', 'ip_titulo_profissional', 'ip_area_atuacao', 'ip_conhecimentos_tecnicos', 'ip_certificacoes', 'ip_outras_certificacoes', 'ip_remuneracao', 'ip_nivel_profissional', 'fei_nivel_academico', 'fei_instituicao_ensino_superior', 'fei_cursos', 'fei_ano_conclusao', 'fei_nivel_ingles', 'fei_nivel_espanhol', 'fei_outro_idioma', 'c

In [2]:
df_applicants = df_applicants.rename({col: f"app_{col}" for col in df_applicants.columns if col != "codigo_candidato"})
df_job = df_job.rename({col: f"job_{col}" for col in df_job.columns if col != "codigo_vaga"})

df = (
    df_prospects
    .rename({"p_codigo": "codigo_candidato"})
    .join(df_applicants, on="codigo_candidato", how="inner")
    .join(df_job, on="codigo_vaga", how="inner")
)

In [5]:
df = remover_pii_e_engineering(df)

In [4]:
from rapidfuzz.fuzz import partial_ratio

# Função auxiliar para gerar indicador booleano
def gerar_indicador(nome: str, col_expr: pl.Expr) -> pl.Expr:
    return (
        pl.when(col_expr.is_not_null() & (col_expr != ""))
        .then(True)
        .otherwise(False)
        .alias(nome)
    )

# Executa fuzzy matching fora do Polars
def comparar_locais_com_fuzzy(locais_candidato: list[str], locais_vaga: list[str], threshold: int = 90) -> bool:
    for loc_candidato in locais_candidato:
        for loc_vaga in locais_vaga:
            if loc_candidato and loc_vaga:
                score = partial_ratio(loc_candidato.upper(), loc_vaga.upper())
                if score >= threshold:
                    return True
    return False


def remover_pii_e_engineering(df: pl.DataFrame) -> pl.DataFrame:
    pii_cols = [
        "p_nome",
        "app_ib_telefone_recado",
        "app_ib_telefone",
        "app_ib_email",
        "app_ip_nome",
        "app_ip_cpf",
        "app_ip_email",
        "app_ip_email_secundario",
        "app_ip_data_nascimento",
        "app_ip_telefone_celular",
        "app_ip_telefone_recado",
        "app_ip_endereco",
        "app_ip_skype",
        "app_ip_url_linkedin",
        "app_ip_facebook",
    ]

    # Adiciona indicadores binários
    df = df.with_columns([
        gerar_indicador("ind_app_telefone", pl.col("app_ib_telefone")),
        gerar_indicador("ind_app_email", pl.col("app_ib_email")),
        gerar_indicador("ind_app_linkedin", pl.col("app_ip_url_linkedin")),
        gerar_indicador("ind_app_endereco", pl.col("app_ip_endereco")),
        gerar_indicador("ind_app_facebook", pl.col("app_ip_facebook")),
    ])

    # Extrai registros para processamento externo
    registros = df.select([
        "app_ib_local", "app_ip_endereco",
        "job_pv_estado", "job_pv_cidade", "job_pv_bairro", "job_pv_regiao", "job_pv_local_trabalho"
    ]).to_dicts()

    # Gera coluna fuzzy de localidade
    matches = []
    for row in registros:
        locais_candidato = [row["app_ib_local"], row["app_ip_endereco"]]
        locais_vaga = [
            row["job_pv_estado"], row["job_pv_cidade"], row["job_pv_bairro"],
            row["job_pv_regiao"], row["job_pv_local_trabalho"]
        ]
        matches.append(comparar_locais_com_fuzzy(locais_candidato, locais_vaga, threshold=70))

    # Adiciona coluna ao DataFrame final
    df = df.with_columns([
        pl.Series("ind_mesma_localidade", matches)
    ])

    # Remove colunas PII
    df = df.drop([col for col in pii_cols if col in df.columns])

    return df

In [5]:
#df.select("app_ip_data_aceite").unique()

df = df.with_columns(
    pl.col("app_ip_data_aceite").str.strptime(pl.Datetime, "%d/%m/%Y %H:%M", strict=False).alias("app_ip_data_aceite_2")
)

df.select("app_ip_data_aceite").head()

app_ip_data_aceite
str
"""CADASTRO ANTERIOR AO REGISTRO DE ACEITE"""
"""CADASTRO ANTERIOR AO REGISTRO DE ACEITE"""
"""CADASTRO ANTERIOR AO REGISTRO DE ACEITE"""
"""CADASTRO ANTERIOR AO REGISTRO DE ACEITE"""
"""25/11/2022 11:04"""


In [ ]:
df.select("codigo_candidato").unique().height #10522 #23457 job_b_valor_venda job_b_valor_compra_1 job_b_valor_compra_2

23457

In [ ]:

df.filter(pl.col('codigo_vaga')=='10') #job_ib_limit                                        e_esperado_para_contratacao

codigo_vaga,titulo,modalidade,p_nome,codigo_candidato,p_situacao_candidado,p_data_candidatura,p_ultima_atualizacao,p_comentario,p_recrutador,app_ib_telefone_recado,app_ib_telefone,app_ib_objetivo_profissional,app_ib_data_criacao,app_ib_inserido_por,app_ib_email,app_ib_local,app_ib_sabendo_de_nos_por,app_ib_data_atualizacao,app_ib_codigo_profissional,app_ib_nome,app_ip_data_aceite,app_ip_download_cv,app_ip_nome,app_ip_cpf,app_ip_fonte_indicacao,app_ip_email,app_ip_email_secundario,app_ip_data_nascimento,app_ip_telefone_celular,app_ip_telefone_recado,app_ip_sexo,app_ip_estado_civil,app_ip_pcd,app_ip_endereco,app_ip_skype,app_ip_url_linkedin,app_ip_facebook,app_ip_titulo_profissional,app_ip_area_atuacao,app_ip_conhecimentos_tecnicos,app_ip_certificacoes,app_ip_outras_certificacoes,app_ip_remuneracao,app_ip_nivel_profissional,app_fei_nivel_academico,app_fei_instituicao_ensino_superior,app_fei_cursos,app_fei_ano_conclusao,app_fei_nivel_ingles,app_fei_nivel_espanhol,app_fei_outro_idioma,app_cv_pt,app_cv_en,job_ib_data_requicisao,job_ib_limite_esperado_para_contratacao,job_ib_titulo_vaga,job_ib_vaga_sap,job_ib_cliente,job_ib_solicitante_cliente,job_ib_empresa_divisao,job_ib_requisitante,job_ib_analista_responsavel,job_ib_tipo_contratacao,job_ib_prazo_contratacao,job_ib_objetivo_vaga,job_ib_prioridade_vaga,job_ib_origem_vaga,job_ib_superior_imediato,job_ib_nome,job_ib_telefone,job_ib_data_inicial,job_ib_data_final,job_pv_pais,job_pv_estado,job_pv_cidade,job_pv_bairro,job_pv_regiao,job_pv_local_trabalho,job_pv_vaga_especifica_para_pcd,job_pv_faixa_etaria,job_pv_horario_trabalho,job_pv_nivel_profissional,job_pv_nivel_academico,job_pv_nivel_ingles,job_pv_nivel_espanhol,job_pv_outro_idioma,job_pv_areas_atuacao,job_pv_principais_atividades,job_pv_competencia_tecnicas_e_comportamentais,job_pv_demais_observacoes,job_pv_viagens_requeridas,job_pv_equipamentos_necessarios,job_pv_habilidades_comportamentais_necessarias,job_b_valor_venda,job_b_valor_compra_1,job_b_valor_compra_2
str,str,str,str,str,str,date,date,str,str,str,str,str,datetime[μs],str,str,str,str,datetime[μs],str,str,str,str,str,str,str,str,str,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""10""","""OPERADOR DE MONITORAÇÃO DE SWC""","""""","""DANIELA MACHADO""","""12622""","""ENCAMINHADO AO REQUISITANTE""",2018-12-07,2018-12-07,"""""","""HELENA BARROS""","""""","""11 96264-8925""","""""",2018-12-07 15:27:05,"""HELENA BARROS""","""DANIELA_MACHADO@GMAIL.COM""","""""","""""",2018-12-07 15:27:05,"""12622""","""DANIELA MACHADO""","""CADASTRO ANTERIOR AO REGISTRO DE ACEITE""",null,"""DANIELA MACHADO""","""""",""":""","""DANIELA_MACHADO@GMAIL.COM""","""""",null,"""11 96264-8925""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",null,null,null,"""""","""""","""-""","""DATA NASC.: 02/02/1999, SOLTEIRA, BRASILEIRA, EXPERIÊNCIA PROFISSIONAL ALGAR TI CONSULTORIA S/A MAIO/2018 - ATUALMENTE SUPORTE EM TI DE 1º NÍVEL ALOCADO NO CLIENTE VIA VAREJO CASAS BAHIA E PONTO FRIO TROUBLESHOOTING EM SERVIDORES IBM E DELL SUPORTE À CONTROLADORES DE ACESSO, RELÓGIO DE PONTO - TELEMÁTICA, TERMINAIS DE ACESSO, DISKLESS E PDV TEF, ECF, SAT, PINPAD, SISTEMAS PROPRIETÁRIOS, APLICAÇÕES WEB/LINUX SUPORTE A IMPRESSORAS MULTIFUNCIONAIS LEXMARK, IMPRESSORAS FISCAIS E NÃO FISCAIS, ZEB…","""""",2018-12-05,2018-12-01,"""OPERADOR DE MONITORAÇÃO DE SWC""","""NÃO""","""WALTERS GROUP""","""LEVI DA LUZ""","""DECISION SÃO PAULO""","""CECÍLIA FREITAS""","""HELENA BARROS""","""CLT FULL""","""INDETERMINADO""","""CONTRATAÇÃO""","""MÉDIA: MÉDIA COMPLEXIDADE 6 A 10 DIAS""","""""","""SUPERIOR IMEDIATO:""","""FELIPE""","""""",null,null,"""BRASIL""","""SÃO PAULO""","""SÃO PAULO""","""BARRA FUNDA""","""OESTE""","""2000""","""NÃO""","""DE: 18 

In [ ]:
from rapidfuzz.fuzz import partial_ratio

def remover_pii_e_engineering(df: pl.DataFrame) -> pl.DataFrame:
    pii_cols = [
        "p_nome",
        "app_ib_telefone_recado",
        "app_ib_telefone",
        "app_ib_email",
        "app_ip_nome",
        "app_ip_cpf",
        "app_ip_email",
        "app_ip_email_secundario",
        "app_ip_data_nascimento",
        "app_ip_telefone_celular",
        "app_ip_telefone_recado",
        "app_ip_endereco",
        "app_ip_skype",
        "app_ip_url_linkedin",
        "app_ip_facebook",
    ]

    # Função auxiliar para gerar indicador booleano
    def gerar_indicador(nome: str, col_expr: pl.Expr) -> pl.Expr:
        return (
            pl.when(col_expr.is_not_null() & (col_expr != ""))
            .then(True)
            .otherwise(False)
            .alias(nome)
        )

    # Adiciona indicadores binários
    df = df.with_columns([
        gerar_indicador("ind_app_telefone", pl.col("app_ib_telefone")),
        gerar_indicador("ind_app_email", pl.col("app_ib_email")),
        gerar_indicador("ind_app_linkedin", pl.col("app_ip_url_linkedin")),
        gerar_indicador("ind_app_endereco", pl.col("app_ip_endereco")),
        gerar_indicador("ind_app_facebook", pl.col("app_ip_facebook")),
    ])

    # Executa fuzzy matching fora do Polars
    def comparar_locais_com_fuzzy(locais_candidato: list[str], locais_vaga: list[str], threshold: int = 90) -> bool:
        for loc_candidato in locais_candidato:
            for loc_vaga in locais_vaga:
                if loc_candidato and loc_vaga:
                    score = partial_ratio(loc_candidato.upper(), loc_vaga.upper())
                    if score >= threshold:
                        return True
        return False

    # Extrai registros para processamento externo
    registros = df.select([
        "app_ib_local", "app_ip_endereco",
        "job_pv_estado", "job_pv_cidade", "job_pv_bairro", "job_pv_regiao", "job_pv_local_trabalho"
    ]).to_dicts()

    # Gera coluna fuzzy de localidade
    matches = []
    for row in registros:
        locais_candidato = [row["app_ib_local"], row["app_ip_endereco"]]
        locais_vaga = [
            row["job_pv_estado"], row["job_pv_cidade"], row["job_pv_bairro"],
            row["job_pv_regiao"], row["job_pv_local_trabalho"]
        ]
        matches.append(comparar_locais_com_fuzzy(locais_candidato, locais_vaga, threshold=70))

    # Adiciona coluna ao DataFrame final
    df = df.with_columns([
        pl.Series("ind_mesma_localidade", matches)
    ])

    # Remove colunas PII
    #df = df.drop([col for col in pii_cols if col in df.columns])

    return df

df = remover_pii_e_engineering(df)

In [4]:
df.group_by("ind_mesma_localidade").agg(pl.len().alias("count"))

ind_mesma_localidade,count
bool,u32
false,29087
true,15984


In [8]:
[a for a in df.columns if 'prioridade' in a]

['job_ib_prioridade_vaga']

In [16]:
def classificar_prioridade_vaga(df: pl.DataFrame) -> pl.DataFrame:
    prioridade_col = (
        pl.col("job_ib_prioridade_vaga")
        .fill_null("")
        .str.strip_chars()
        .str.to_uppercase()
    )

    return df.with_columns(
        pl.when(
            prioridade_col.str.contains(r"(ALTA|HIGH|URGENTE|URGENCY)")
        ).then(pl.lit("ALTA"))
        .when(
            prioridade_col.str.contains(r"(MEDIA|MÉDIA|MEDIUM|INTERMEDIARIA|INTERMEDIÁRIA)")
        ).then(pl.lit("MEDIA"))
        .when(
            prioridade_col.str.contains(r"(BAIXA|LOW|BAIXA PRIORIDADE)")
        ).then(pl.lit("BAIXA"))
        .otherwise(pl.lit("DESCONHECIDO"))
        .alias("job_ib_prioridade_vaga")
    )

df = classificar_prioridade_vaga(df)

In [17]:
distinct_prioridade = (
    df.select("job_ib_prioridade_vaga")
      .unique()
)

print("\nÁreas de prioridade:")
print(distinct_prioridade)


Áreas de prioridade:
shape: (4, 1)
┌────────────────────────┐
│ job_ib_prioridade_vaga │
│ ---                    │
│ str                    │
╞════════════════════════╡
│ MEDIA                  │
│ ALTA                   │
│ DESCONHECIDO           │
│ BAIXA                  │
└────────────────────────┘


In [20]:
distinct_prioridade = (
    df.select("job_ib_superior_imediato")
      .unique()
)

print("\nsuperior:")
print(distinct_prioridade)

df.select([
    pl.col("codigo_vaga").filter(pl.col("job_ib_superior_imediato").is_null()).n_unique().alias("count_vagas_sem_superior"),
    pl.col("codigo_vaga").filter(pl.col("job_ib_superior_imediato").is_not_null()).n_unique().alias("count_vagas_com_superior")
])



superior:
shape: (2, 1)
┌──────────────────────────┐
│ job_ib_superior_imediato │
│ ---                      │
│ str                      │
╞══════════════════════════╡
│ SUPERIOR IMEDIATO:       │
│ null                     │
└──────────────────────────┘


count_vagas_sem_superior,count_vagas_com_superior
u32,u32
27,10495


In [21]:

distinct_prioridade = (
    df.select("p_situacao_candidado")
      .unique()
)

print("\nSituacao do candidato:")
print(distinct_prioridade)



Situacao do candidato:
shape: (21, 1)
┌────────────────────────────────┐
│ p_situacao_candidado           │
│ ---                            │
│ str                            │
╞════════════════════════════════╡
│ PROSPECT                       │
│ DESISTIU DA CONTRATAÇÃO        │
│ INSCRITO                       │
│ NÃO APROVADO PELO REQUISITANTE │
│ ENCAMINHADO AO REQUISITANTE    │
│ ENTREVISTA COM CLIENTE         │
│ NÃO APROVADO PELO RH           │
│ ENTREVISTA TÉCNICA             │
│ RECUSADO                       │
│ DESISTIU                       │
│ NÃO APROVADO PELO CLIENTE      │
│ DOCUMENTAÇÃO PJ                │
│ EM AVALIAÇÃO PELO RH           │
│ DOCUMENTAÇÃO CLT               │
│ SEM INTERESSE NESTA VAGA       │
│ ENCAMINHAR PROPOSTA            │
│ APROVADO                       │
│ DOCUMENTAÇÃO COOPERADO         │
│ CONTRATADO COMO HUNTING        │
│ CONTRATADO PELA DECISION       │
│ PROPOSTA ACEITA                │
└────────────────────────────────┘


In [6]:
def extrair_lista(df: pl.DataFrame, coluna: str, spliter: str) -> pl.DataFrame:
    """
    Cria nova coluna com lista padronizada e limpa, removendo elementos vazios e substituindo por 'DESCONHECIDO' se necessário.
    """
    # Pré-processamento e transformação dos elementos
    col_limpa = (
        pl.col(coluna)
        .fill_null("")
        .str.to_lowercase()
        .str.replace_all(r"\s+", " ")
        .str.strip_chars()
        .str.split(spliter)
        .list.eval(
            pl.element()
            .str.strip_chars()
            .str.strip_chars(" ")
            .str.to_uppercase()
        )
    )

    # Lista com elementos vazios removidos
    col_filtrada = col_limpa.list.eval(pl.element().filter(pl.element() != ""))

    return df.with_columns(
        pl.when(
            (col_filtrada.list.len() == 0)
        ).then(
            pl.lit(["DESCONHECIDO"])
        ).otherwise(
            col_filtrada
        ).alias(f"{coluna}_cleaned_list")
    )

list_to_extract = [
    ("job_ib_tipo_contratacao", ","),
    ("job_pv_areas_atuacao", "-"),
]

# Aplicar a transformação para cada coluna na lista
for coluna, spliter in list_to_extract:
    df = extrair_lista(df, coluna, spliter)

# Exemplo de visualização
print(
    df.select([
        "job_ib_tipo_contratacao", "job_ib_tipo_contratacao_cleaned_list",
        "job_pv_areas_atuacao", "job_pv_areas_atuacao_cleaned_list",
    ])
    .head()
)



shape: (5, 4)
┌─────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬─────────────────────────────────────────┐
│ job_ib_tipo_contratacao ┆ job_ib_tipo_contratacao_cleaned_list ┆ job_pv_areas_atuacao                 ┆ job_pv_areas_atuacao_cleaned_list       │
│ ---                     ┆ ---                                  ┆ ---                                  ┆ ---                                     │
│ str                     ┆ list[str]                            ┆ str                                  ┆ list[str]                               │
╞═════════════════════════╪══════════════════════════════════════╪══════════════════════════════════════╪═════════════════════════════════════════╡
│ PJ/AUTÔNOMO             ┆ ["PJ/AUTÔNOMO"]                      ┆ TI - DESENVOLVIMENTO/PROGRAMAÇÃO-    ┆ ["TI", "DESENVOLVIMENTO/PROGRAMAÇÃO"]   │
│ PJ/AUTÔNOMO             ┆ ["PJ/AUTÔNOMO"]                      ┆ TI - DESENVOLVIMENTO/PROGRAMAÇÃ

In [ ]:
# -------------------------------------------------------------------------
# Função de análise estruturada
# -------------------------------------------------------------------------

def analisar_vaga(row: dict) -> str:
    def extrair_texto(row, key):
        valor = row.get(key, "")
        if isinstance(valor, list):
            valor = valor[0] if valor else ""
        return str(valor).strip() if valor else ""

    # Prepara prompt
    return f"""
Você é um especialista em Recursos Humanos com foco em análise de descrições de vagas. 
Sua tarefa é extrair e estruturar todas as informações relevantes a partir de uma descrição de vaga de forma completa e minuciosa para gerar um "perfil estruturado da vaga". 
Identificar e extrair informações estruturadas sobre o candidato;
Resumir cada característica solicitada em 1 ou 2 palavras (máximo) de cada item;
Retorne o maximo de informações possíveis, mesmo que não estejam explicitamente mencionadas e neste caso retorne ao menos 10 caracteristicas, mas sejam inferíveis a partir do contexto.

Caso algum dado não esteja presente, use “Não mencionado”.
Título: {extrair_texto(row, 'job_ib_titulo_vaga')}
Atividades: {extrair_texto(row, 'job_pv_principais_atividades')}
Competências: {extrair_texto(row, 'job_pv_competencia_tecnicas_e_comportamentais')}
Observações: {extrair_texto(row, 'job_pv_demais_observacoes')}
Habilidades comportamentais: {extrair_texto(row, 'job_pv_habilidades_comportamentais_necessarias')}

Com base nessas informações, responda somente este JSON, preenchendo todos os campos minuciosamente, sem deixar campos genéricos:

{{
  "ferramentas_tecnologicas": ["Liste todas as ferramentas, plataformas e tecnologias citadas ou implícitas na vaga."],
  "competencias_comportamentais": ["Liste todas as soft skills e atitudes esperadas, mesmo que indiretamente mencionadas."],
  "competencias_tecnicas": ["Liste o máximo de competências técnicas (hard skills) exigidas ou desejáveis para a função."],
  "senioridade_sugerida": "Escolha uma: Estágio / Júnior / Pleno / Sênior / Indefinida, com base na vaga.",
  "nivel_de_clareza": "Classifique a clareza da vaga: Alto / Médio / Baixo, de acordo com a objetividade e detalhamento.",
  "formacao_academica": "Informe a formação esperada (explícita ou implícita), como: 'Ensino superior em Administração', 'Tecnólogo em TI', ou 'Não especificado'."
}}

Atenção:
- Seja fiel ao texto; não invente informações.
- Ao inferir, fundamente sua resposta de acordo com padrões de mercado (exemplo: domínio de ferramentas avançadas pode indicar senioridade maior).
- Não inclua explicações fora do JSON.
    """.strip()


def avaliar_candidato(row: dict) -> str:
    def extrair_texto(row, key):
        valor = row.get(key, "")
        if isinstance(valor, list):
            valor = valor[0] if valor else ""
        return str(valor).strip() if valor else ""

    # Prepara prompt
    return f"""
Você é um especialista em análise de currículos com foco em recrutamento técnico e comportamental para posições de TI e correlatas. Sua tarefa é:
Ler atentamente o conteúdo abaixo, extraído do currículo (texto livre).
Identificar e extrair informações estruturadas sobre o candidato, considerando tanto menções explícitas quanto inferências razoáveis a partir do texto.
Caso algum dado não esteja presente, utilize a expressão “Não mencionado”.
Responda de forma objetiva, clara e livre de interpretações subjetivas.
Retorne o maximo de informações possíveis, mesmo que não estejam explicitamente mencionadas e neste caso retorne ao menos 10 caracteristicas, mas sejam inferíveis a partir do contexto.

Conteúdo do currículo (texto livre):
{extrair_texto(row, 'app_cv_pt')}

Devolva apenas um objeto em formato JSON conforme o modelo abaixo, preenchendo todos os campos solicitados:

json
{{
    "ferramentas_tecnologicas": ["Liste apenas ferramentas, linguagens ou plataformas citadas ou inferidas, como Excel, Python, SQL, Power BI, etc."],
    "competencias_tecnicas": ["Liste competências técnicas (ex: análise de dados, desenvolvimento backend, machine learning, etc.)"],
    "competencias_comportamentais": ["Liste competências comportamentais (ex: trabalho em equipe, comunicação, proatividade, etc.)"],
    "experiencia_anos": "Informe a estimativa da experiência do candidato em anos, se possível, a partir do texto. Se não houver dado suficiente, responda 'Não mencionado'.",
    "senioridade_aparente": "Classifique como: Estágio / Júnior / Pleno / Sênior / Indefinida, conforme as evidências do texto.",
    "formacao_academica": "True or false",
    "nivel_formacao": "Informe o nível mais alto mencionado (ex: Ensino Médio, Técnico, Superior, Pós-graduação). Caso não haja menção, use 'Não mencionado'",
    "area_formacao": "área de maneira macro. Caso não haja menção, use 'Não mencionado'",
    "nivel_ingles": "Classifique como: Básico / Intermediário / Avançado / Fluente / Não mencionado"
}}
Atenção:
- Seja fiel ao texto; não invente informações.
- Ao inferir, fundamente sua resposta de acordo com padrões de mercado (exemplo: domínio de ferramentas avançadas pode indicar senioridade maior).
- Não inclua explicações fora do JSON.
    """.strip()




# -------------------------------------------------------------------------
# Executar análise em uma linha do DataFrame
# -------------------------------------------------------------------------

linha_index = 0
job_row = df[linha_index].to_dict(as_series=False)
resultado_estruturado = analisar_vaga(job_row)
# Exibir resultado
print(resultado_estruturado)

resultado_estruturado = avaliar_candidato(job_row)
# Exibir resultado
print(resultado_estruturado)

Você é um especialista em Recursos Humanos com foco em análise de descrições de vagas. 
Sua tarefa é extrair e estruturar todas as informações relevantes a partir de uma descrição de vaga de forma completa e minuciosa para gerar um "perfil estruturado da vaga". 
Identificar e extrair informações estruturadas sobre o candidato;
Resumir cada característica solicitada em 1 ou 2 palavras (máximo) de cada item;
Retorne o maximo de informações possíveis, mesmo que não estejam explicitamente mencionadas e neste caso retorne ao menos 10 caracteristicas, mas sejam inferíveis a partir do contexto.

Caso algum dado não esteja presente, use “Não mencionado”.
Título: CONSULTOR CONTROL M
Atividades: - EXPERIÊNCIA COMPROVADA EM PROJETOS DE CONTROL-M
Competências: - EXPERIÊNCIA COMPROVADA EM PROJETOS DE CONTROL-M
Observações: CONTRATAÇÃO PJ PROJETO PONTUAL DE 2 A 3 MESES CLIENTE: CAPGEMINI PROJETO: LIGHT / RIO DEJANEIRO LOCAL DE TRABALHO: HIBRIDA: REMOTA E PRESENCIAL NA LIGHT NO RJ QUANDO SOLICITADO
H

In [ ]:
def calcular_score(candidato, vaga):
    def calcular_match(lista_candidato, lista_vaga):
        if not lista_vaga:
            return 1.0
        intersecao = set(lista_candidato).intersection(set(lista_vaga))
        return len(intersecao) / len(lista_vaga)

    match_ferramentas = calcular_match(candidato["ferramentas_tecnologicas"], vaga["ferramentas_tecnologicas"])
    match_competencias_tecnicas = calcular_match(candidato["competencias_tecnicas"], vaga["competencias_tecnicas"])
    senioridade_ok = 1.0 if candidato["senioridade_aparente"] == vaga["senioridade_sugerida"] else 0.5
    formacao_ok = 1.0 if candidato["formacao_academica"] else 0.0
    match_comportamental = calcular_match(candidato["competencias_comportamentais"], vaga["competencias_comportamentais"])

    score_tecnico = (
        match_ferramentas * 0.4 +
        match_competencias_tecnicas * 0.4 +
        senioridade_ok * 0.1 +
        formacao_ok * 0.1
    )

    score_cultural = match_comportamental
    score_final = round((score_tecnico * 0.7 + score_cultural * 0.3) * 100, 2)

    return {
        "score_tecnico": round(score_tecnico * 100, 2),
        "score_cultural": round(score_cultural * 100, 2),
        "score_final_skils": score_final
    }


In [7]:
pv_competencia_tecnicas_e_comportamentais = df["pv_competencia_tecnicas_e_comportamentais"].to_list()

print(pv_competencia_tecnicas_e_comportamentais[2])

REQUISITOS MANDATÓRIOS: O CONHECIMENTOS TÉCNICOS: CONHECIMENTO SQL, E ALGUM CONHECIMENTO DE SAP SD O COMPETÊNCIAS INTERPESSOAIS: RELACIONAMENTO INTERPESSOAL, FOCO NO CLIENTE, TRABALHO EM EQUIPE, EXCELENTE COMUNICAÇÃO, E ADAPTABILIDADE. REQUISITOS DESEJÁVEIS: O IDIOMAS: ESPANHOL NÍVEL INTERMEDIÁRIO.


In [7]:
# Explode e pegue os valores distintos da primeira coluna
distinct_contratacao = (
    df.select("job_ib_tipo_contratacao_cleaned_list")
      .explode("job_ib_tipo_contratacao_cleaned_list")
      .unique()
)

# Explode e pegue os valores distintos da segunda coluna
distinct_areas_atuacao = (
    df.select("job_pv_areas_atuacao_cleaned_list")
      .explode("job_pv_areas_atuacao_cleaned_list")
      .unique()
)

# Aumentar largura da tabela e o comprimento de strings, se necessário
pl.Config.set_tbl_width_chars(200)       # largura total da tabela no terminal
pl.Config.set_fmt_str_lengths(2000)      # número máximo de caracteres por célula
pl.Config.set_tbl_rows(-1)  # -1 remove o limite

print("Tipos de Contratação:")
print(distinct_contratacao)

print("\nÁreas de Atuação:")
print(distinct_areas_atuacao)


Tipos de Contratação:
shape: (8, 1)
┌──────────────────────────────────────┐
│ job_ib_tipo_contratacao_cleaned_list │
│ ---                                  │
│ str                                  │
╞══════════════════════════════════════╡
│ CLT FULL                             │
│ CLT COTAS                            │
│ CANDIDATO PODERÁ ESCOLHER            │
│ ESTAGIÁRIO                           │
│ HUNTING                              │
│ COOPERADO                            │
│                                      │
│ PJ/AUTÔNOMO                          │
└──────────────────────────────────────┘

Áreas de Atuação:
shape: (28, 1)
┌─────────────────────────────────────┐
│ job_pv_areas_atuacao_cleaned_list   │
│ ---                                 │
│ str                                 │
╞═════════════════════════════════════╡
│ COMERCIAL                           │
│ TELECOM                             │
│                                     │
│ RELACIONAMENTO TÉCNICO            

In [ ]:
df.filter(pl.col('codigo_vaga')=="10").head()

codigo_vaga,titulo,modalidade,p_nome,codigo_candidato,p_situacao_candidado,p_data_candidatura,p_ultima_atualizacao,p_comentario,p_recrutador,app_ib_telefone_recado,app_ib_telefone,app_ib_objetivo_profissional,app_ib_data_criacao,app_ib_inserido_por,app_ib_email,app_ib_local,app_ib_sabendo_de_nos_por,app_ib_data_atualizacao,app_ib_codigo_profissional,app_ib_nome,app_ip_data_aceite,app_ip_download_cv,app_ip_nome,app_ip_cpf,app_ip_fonte_indicacao,app_ip_email,app_ip_email_secundario,app_ip_data_nascimento,app_ip_telefone_celular,app_ip_telefone_recado,app_ip_sexo,app_ip_estado_civil,app_ip_pcd,app_ip_endereco,app_ip_skype,app_ip_url_linkedin,app_ip_facebook,app_ip_titulo_profissional,app_ip_area_atuacao,app_ip_conhecimentos_tecnicos,app_ip_certificacoes,app_ip_outras_certificacoes,app_ip_remuneracao,app_ip_nivel_profissional,app_fei_nivel_academico,app_fei_instituicao_ensino_superior,app_fei_cursos,app_fei_ano_conclusao,app_fei_nivel_ingles,app_fei_nivel_espanhol,app_fei_outro_idioma,app_cv_pt,app_cv_en,job_ib_data_requicisao,job_ib_limite_esperado_para_contratacao,job_ib_titulo_vaga,job_ib_vaga_sap,job_ib_cliente,job_ib_solicitante_cliente,job_ib_empresa_divisao,job_ib_requisitante,job_ib_analista_responsavel,job_ib_tipo_contratacao,job_ib_prazo_contratacao,job_ib_objetivo_vaga,job_ib_prioridade_vaga,job_ib_origem_vaga,job_ib_superior_imediato,job_ib_nome,job_ib_telefone,job_ib_data_inicial,job_ib_data_final,job_pv_pais,job_pv_estado,job_pv_cidade,job_pv_bairro,job_pv_regiao,job_pv_local_trabalho,job_pv_vaga_especifica_para_pcd,job_pv_faixa_etaria,job_pv_horario_trabalho,job_pv_nivel_profissional,job_pv_nivel_academico,job_pv_nivel_ingles,job_pv_nivel_espanhol,job_pv_outro_idioma,job_pv_areas_atuacao,job_pv_principais_atividades,job_pv_competencia_tecnicas_e_comportamentais,job_pv_demais_observacoes,job_pv_viagens_requeridas,job_pv_equipamentos_necessarios,job_pv_habilidades_comportamentais_necessarias,job_b_valor_venda,job_b_valor_compra_1,job_b_valor_compra_2
str,str,str,str,str,str,date,date,str,str,str,str,str,date,str,str,str,str,date,str,str,date,str,str,str,str,str,str,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""4530""","""CONSULTOR CONTROL M""","""""","""JOSÉ VIEIRA""","""25632""","""ENCAMINHADO AO REQUISITANTE""",null,null,"""ENCAMINHADO PARA - PJ R 72,00/HORA""","""ANA LÍVIA MOREIRA""","""""","""21 93485-6494""","""""",null,"""ANA LÍVIA MOREIRA""","""JOSÉ_VIEIRA@GMAIL.COM""","""""","""""",null,"""25632""","""JOSÉ VIEIRA""",null,null,"""JOSÉ VIEIRA""","""""",""":""","""JOSÉ_VIEIRA@GMAIL.COM""","""""",null,"""21 93485-6494""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",null,null,null,"""""","""""","""-""","""DADOS PESSOAIS ESTADO CIVIL: CASADO IDADE: 33 ANOS OBJETIVO LEVAR À EMPRESA O MEU CONHECIMENTO, TORNANDO-ME UM MEMBRO A MAIS PARA O SEU DESENVOLVIMENTO. FORMAÇÃO ACADÊMICA UNIVERSIDADE ESTÁCIO DE SÁ RJ MBA EM CIÊNCIA DE DADOS E BIG DATA ANALYTICS - CONCLUÍDO UNIVERSIDADE ESTÁCIO DE SÁ RJ SUPERIOR EM ANÁLISE E DESENVOLVIMENTO DE SISTEMAS - CONCLUÍDO ETE JUSCELINO KUBITSCHEK FAETEC TÉCNICO EM ELETRICIDADE - CONCLUÍDO CERTIFICADOS CERTIFICADO ITIL FOUNDADION V3 CURSOS DE APERFEIÇOAMENTO MONTAGEM…","""""",null,"""00-00-0000""","""CONSULTOR CONTROL M""","""NÃO""","""MORRIS, MORAN AND DODSON""","""DRA. CATARINA MARQUES""","""DECISION SÃO PAULO""","""MARIA LAURA NOGUEIRA""","""ANA LÍVIA MOREIRA""","""PJ/AUTÔNOMO""","""""","""""","""""","""""","""SUPERIOR IMEDIATO:""","""""","""""",null,null,"""BRASIL""","""RIO DE JANEIRO""","""RIO DE JANEIRO""","""""","""""","""2000""","""NÃO""","""DE: ATÉ:""","""""","""PLENO""","""ENSINO SUPERIOR COMPLETO""","""NENHUM""","""NENHUM""","""""","""TI - DESENVOLVIMENTO/PROGRAMAÇÃO-""","""- EXP

: 

In [5]:
[a for a in df.columns if 'data' in a] + ["job_ib_limite_esperado_para_contratacao"]

['p_data_candidatura',
 'app_ib_data_criacao',
 'app_ib_data_atualizacao',
 'app_ip_data_aceite',
 'app_ip_data_nascimento',
 'job_ib_data_requicisao',
 'job_ib_data_inicial',
 'job_ib_data_final',
 'job_ib_limite_esperado_para_contratacao']

In [11]:
df.select([a for a in df.columns if 'data' in a] + ["job_ib_limite_esperado_para_contratacao"]).head()#.to_pandas().describe().T

p_data_candidatura,app_ib_data_criacao,app_ib_data_atualizacao,app_ip_data_aceite,app_ip_data_nascimento,job_ib_data_requicisao,job_ib_data_inicial,job_ib_data_final,job_ib_limite_esperado_para_contratacao
date,datetime[μs],datetime[μs],str,date,date,date,date,date
2021-03-25,2021-03-25 13:36:19,2021-03-25 13:36:19,"""CADASTRO ANTERIOR AO REGISTRO DE ACEITE""",null,2021-03-10,null,null,null
2021-03-22,2021-03-22 22:48:58,2021-03-22 23:36:28,"""CADASTRO ANTERIOR AO REGISTRO DE ACEITE""",1973-10-25,2021-03-10,null,null,null
2021-03-17,2021-03-17 09:41:10,2021-04-12 12:15:57,"""CADASTRO ANTERIOR AO REGISTRO DE ACEITE""",1973-01-31,2021-03-10,null,null,null
2021-03-17,2021-03-17 08:20:21,2021-03-17 08:20:21,"""CADASTRO ANTERIOR AO REGISTRO DE ACEITE""",null,2021-03-10,null,null,null
2021-04-29,2021-04-27 13:12:34,2022-11-25 11:04:15,"""25/11/2022 11:04""",null,2021-03-11,null,null,1970-01-01


In [17]:
def gerar_features_temporais(df: pl.DataFrame) -> pl.DataFrame:
    max_data_candidatura = df.select(pl.col("p_data_candidatura").max()).item()
    hoje = pl.lit(max_data_candidatura)

    def dias(col1, col2, nome, abs_val=False):
        expr = (col1 - col2).dt.total_days()
        if abs_val:
            expr = expr.abs()
        return (
            pl.when(col1.is_not_null() & col2.is_not_null())
            .then(expr)
            .otherwise(-1)
            .alias(nome)
            .clip(0, None)  # ✅ CORRETO
        )

    return df.with_columns([
        dias(pl.col("app_ib_data_criacao"), pl.col("p_data_candidatura"), "tempo_entre_criacao_e_candidatura", abs_val=True),
        dias(pl.col("p_data_candidatura"), pl.col("job_ib_data_inicial"), "tempo_entre_job_inicial_e_candidatura"),
        dias(pl.col("job_ib_limite_esperado_para_contratacao"), pl.col("job_ib_data_requicisao"), "tempo_para_contratacao"),
        (
            pl.when(pl.col("app_ib_data_criacao").is_not_null() & pl.col("app_ip_data_nascimento").is_not_null())
            .then((pl.col("app_ib_data_criacao") - pl.col("app_ip_data_nascimento")).dt.total_days() / 365.25)
            .otherwise(-1)
            .alias("idade_candidato")
            .clip(0, None)
        ),
        dias(pl.col("app_ib_data_atualizacao"), pl.col("app_ib_data_criacao"), "tempo_ultima_atualizacao_aplicacao"),
        dias(pl.col("job_ib_data_final"), pl.col("job_ib_data_inicial"), "tempo_vaga_aberta"),
        (
            pl.when(pl.col("job_ib_data_final").is_not_null())
            .then((pl.col("job_ib_data_final") > hoje).cast(pl.Int8))
            .otherwise(-1)
            .alias("vaga_em_aberto_no_momento")
        ),
        (
            pl.when(pl.col("p_data_candidatura").is_not_null() & pl.col("job_ib_data_inicial").is_not_null())
            .then((pl.col("p_data_candidatura") < pl.col("job_ib_data_inicial")).cast(pl.Int8))
            .otherwise(-1)
            .alias("candidatura_antes_da_abertura_oficial")
        ),
        (
            pl.when(pl.col("job_ib_data_requicisao").is_not_null())
            .then((hoje - pl.col("job_ib_data_requicisao")).dt.total_days())
            .otherwise(-1)
            .alias("dias_desde_requisicao")
            .clip(0, None)
        ),
        (
            pl.when(pl.col("job_ib_data_requicisao").is_not_null())
            .then(((hoje - pl.col("job_ib_data_requicisao")).dt.total_days() < 30).cast(pl.Int8))
            .otherwise(-1)
            .alias("vaga_requisitada_recente")
        )
    ])


df = gerar_features_temporais(df)

df.select([
    "codigo_vaga",
    "codigo_candidato",
    "tempo_entre_criacao_e_candidatura",
    "tempo_entre_job_inicial_e_candidatura",
    "tempo_para_contratacao",
    "tempo_ultima_atualizacao_aplicacao",
    "tempo_vaga_aberta",
    "vaga_em_aberto_no_momento",
    "candidatura_antes_da_abertura_oficial",
    "dias_desde_requisicao",
    "vaga_requisitada_recente",
]).head(10)

codigo_vaga,codigo_candidato,tempo_entre_criacao_e_candidatura,tempo_entre_job_inicial_e_candidatura,tempo_para_contratacao,tempo_ultima_atualizacao_aplicacao,tempo_vaga_aberta,vaga_em_aberto_no_momento,candidatura_antes_da_abertura_oficial,dias_desde_requisicao,vaga_requisitada_recente
str,str,i64,i64,i64,i64,i64,i8,i8,i64,i8
"""4530""","""25632""",0,0,0,0,0,-1,-1,1454,0
"""4530""","""25529""",0,0,0,0,0,-1,-1,1454,0
"""4531""","""25364""",0,0,0,26,0,-1,-1,1454,0
"""4531""","""25360""",0,0,0,0,0,-1,-1,1454,0
"""4533""","""26338""",1,0,0,576,0,-1,-1,1453,0
"""4533""","""24645""",63,0,0,153,0,-1,-1,1453,0
"""4534""","""26361""",0,0,0,0,0,-1,-1,1453,0
"""4534""","""26205""",3,0,0,0,0,-1,-1,1453,0
"""4534""","""26003""",0,0,0,0,0,-1,-1,1453,0


: 

In [12]:
df_prospects.head()

codigo_vaga,titulo,modalidade,p_nome,p_codigo,p_situacao_candidado,p_data_candidatura,p_ultima_atualizacao,p_comentario,p_recrutador
str,str,str,str,str,str,date,date,str,str
"""4530""","""CONSULTOR CONTROL M""","""""","""JOSÉ VIEIRA""","""25632""","""ENCAMINHADO AO REQUISITANTE""",null,null,"""ENCAMINHADO PARA - PJ R 72,00/HORA""","""ANA LÍVIA MOREIRA"""
"""4530""","""CONSULTOR CONTROL M""","""""","""SRTA. ISABELA CAVALCANTE""","""25529""","""ENCAMINHADO AO REQUISITANTE""",null,null,"""ENCAMINHADO PARA - R 6.000,00 CLT FULL , NÃO TEM EMPRESA ABERTA""","""ANA LÍVIA MOREIRA"""
"""4531""","""2021-2607395-PEOPLESOFT APPLICATION ENGINE-DOMAIN CONSULTANT""","""""","""SRA. YASMIN FERNANDES""","""25364""","""CONTRATADO PELA DECISION""",null,null,"""DATA DE INICIO: 12/04/2021""","""JULIANA CASSIANO"""
"""4531""","""2021-2607395-PEOPLESOFT APPLICATION ENGINE-DOMAIN CONSULTANT""","""""","""ALEXIA BARBOSA""","""25360""","""ENCAMINHADO AO REQUISITANTE""",null,null,"""""","""JULIANA CASSIANO"""
"""4533""","""2021-2605708-MICROFOCUS APPLICATION LIFE CYCLE MANAGEMENT - HP QUALITY CENTER-ENGINEERING LEAD""","""""","""ARTHUR ALMEIDA""","""26338""","""CONTRATADO PELA DECISION""",null,null,"""""","""STELLA VIEIRA"""


In [7]:
[a for a in df.columns if 'data' in a] + ["job_ib_limite_esperado_para_contratacao"]


['p_data_candidatura',
 'app_ib_data_criacao',
 'app_ib_data_atualizacao',
 'app_ip_data_aceite',
 'app_ip_data_nascimento',
 'job_ib_data_requicisao',
 'job_ib_data_inicial',
 'job_ib_data_final',
 'job_ib_limite_esperado_para_contratacao']

In [2]:
df_as = pl.read_parquet('../data/feature_store/resultado_final.parquet')

In [7]:
df_as.schema

Schema([('codigo_vaga', String),
        ('titulo', String),
        ('modalidade', String),
        ('codigo_candidato', String),
        ('p_situacao_candidado', String),
        ('p_data_candidatura', Date),
        ('p_ultima_atualizacao', Date),
        ('p_comentario', String),
        ('p_recrutador', String),
        ('app_ib_objetivo_profissional', String),
        ('app_ib_data_criacao', Datetime(time_unit='us', time_zone=None)),
        ('app_ib_inserido_por', String),
        ('app_ib_local', String),
        ('app_ib_sabendo_de_nos_por', String),
        ('app_ib_data_atualizacao', Datetime(time_unit='us', time_zone=None)),
        ('app_ib_codigo_profissional', String),
        ('app_ib_nome', String),
        ('app_ip_data_aceite', String),
        ('app_ip_download_cv', String),
        ('app_ip_fonte_indicacao', String),
        ('app_ip_sexo', String),
        ('app_ip_estado_civil', String),
        ('app_ip_pcd', String),
        ('app_ip_titulo_profissional', Strin

In [3]:
df_as.head()

codigo_vaga,titulo,modalidade,codigo_candidato,p_situacao_candidado,p_data_candidatura,p_ultima_atualizacao,p_comentario,p_recrutador,app_ib_objetivo_profissional,app_ib_data_criacao,app_ib_inserido_por,app_ib_local,app_ib_sabendo_de_nos_por,app_ib_data_atualizacao,app_ib_codigo_profissional,app_ib_nome,app_ip_data_aceite,app_ip_download_cv,app_ip_fonte_indicacao,app_ip_sexo,app_ip_estado_civil,app_ip_pcd,app_ip_titulo_profissional,app_ip_area_atuacao,app_ip_conhecimentos_tecnicos,app_ip_certificacoes,app_ip_outras_certificacoes,app_ip_remuneracao,app_ip_nivel_profissional,app_fei_nivel_academico,app_fei_instituicao_ensino_superior,app_fei_cursos,app_fei_ano_conclusao,app_fei_nivel_ingles,app_fei_nivel_espanhol,app_fei_outro_idioma,app_cv_pt,app_cv_en,job_ib_data_requicisao,job_ib_limite_esperado_para_contratacao,job_ib_titulo_vaga,job_ib_vaga_sap,job_ib_cliente,job_ib_solicitante_cliente,job_ib_empresa_divisao,job_ib_requisitante,job_ib_analista_responsavel,job_ib_tipo_contratacao,job_ib_prazo_contratacao,job_ib_objetivo_vaga,job_ib_prioridade_vaga,job_ib_origem_vaga,job_ib_superior_imediato,job_ib_nome,job_ib_telefone,job_ib_data_inicial,job_ib_data_final,job_pv_pais,job_pv_estado,job_pv_cidade,job_pv_bairro,job_pv_regiao,job_pv_local_trabalho,job_pv_vaga_especifica_para_pcd,job_pv_faixa_etaria,job_pv_horario_trabalho,job_pv_nivel_profissional,job_pv_nivel_academico,job_pv_nivel_ingles,job_pv_nivel_espanhol,job_pv_outro_idioma,job_pv_areas_atuacao,job_pv_principais_atividades,job_pv_competencia_tecnicas_e_comportamentais,job_pv_demais_observacoes,job_pv_viagens_requeridas,job_pv_equipamentos_necessarios,job_pv_habilidades_comportamentais_necessarias,job_b_valor_venda,job_b_valor_compra_1,job_b_valor_compra_2,ind_app_telefone,ind_app_email,ind_app_linkedin,ind_app_endereco,ind_app_facebook,ind_mesma_localidade,job_ib_tipo_contratacao_cleaned_list,job_pv_areas_atuacao_cleaned_list,tempo_entre_criacao_e_candidatura,tempo_entre_job_inicial_e_candidatura,tempo_para_contratacao,tempo_ultima_atualizacao_aplicacao,tempo_vaga_aberta,vaga_em_aberto_no_momento,candidatura_antes_da_abertura_oficial,dias_desde_requisicao,vaga_requisitada_recente,job_ind_beneficios_declarados,match_pcd,job_ferramentas_tecnologicas,job_competencias_comportamentais,job_competencias_tecnicas,job_experiencia_anos,job_senioridade_aparente,job_formacao_academica,job_nivel_formacao,job_area_formacao,app_principais_ferramentas_tecnologicas,app_ferramentas_tecnologicas,app_competencias_tecnicas,app_competencias_comportamentais,app_experiencia_anos,app_senioridade_aparente,app_formacao_academica,app_nivel_formacao,app_area_formacao
str,str,str,str,str,date,date,str,str,str,datetime[μs],str,str,str,datetime[μs],str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,bool,bool,bool,bool,bool,list[str],list[str],i64,i64,i64,i64,i64,i64,i64,i64,i64,bool,bool,list[str],list[str],list[str],str,str,str,str,str,list[str],list[str],list[str],list[str],str,str,str,str,str
"""4530""","""CONSULTOR CONTROL M""","""""","""25632""","""ENCAMINHADO AO REQUISITANTE""",2021-03-25,2021-03-25,"""ENCAMINHADO PARA - PJ R 72,00/HORA""","""ANA LÍVIA MOREIRA""","""""",2021-03-25 13:36:19,"""ANA LÍVIA MOREIRA""","""""","""""",2021-03-25 13:36:19,"""25632""","""JOSÉ VIEIRA""","""CADASTRO ANTERIOR AO REGISTRO DE ACEITE""",null,""":""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",null,null,null,"""""","""""","""-""","""DADOS PESSOAIS ESTADO CIVIL: CASADO IDADE: 33 ANOS OBJETIVO LEVAR À EMPRESA O MEU CONHECIMENTO, TORNANDO-ME UM MEMBRO A MAIS PARA O SEU DESENVOLVIMENTO. FORMAÇÃO ACADÊMICA UNIVERSIDADE ESTÁCIO DE SÁ RJ MBA EM CIÊNCIA DE DADOS E BIG DATA ANALYTICS - CONCLUÍDO UNIVERSIDADE ESTÁCIO DE SÁ RJ SUPERIOR EM ANÁLISE E DESENVOLVIMENTO DE SIS